This script uses "optimal play" in order to see the win/loss count of perfect play without counting cards

In [24]:
import random
import json


with open('optimal_play.json') as json_file:
    optimal_play_dict = json.load(json_file)


In [98]:
#create set of cards

def generate_deck(num_of_decks):
    deck_of_cards = [
        "AS", "KS", "QS", "JS", "10S", "9S", "8S", "7S", "6S", "5S", "4S", "3S", "2S",
        "AD", "KD", "QD", "JD", "10D", "9D", "8D", "7D", "6D", "5D", "4D", "3D", "2D",
        "AC", "KC", "QC", "JC", "10C", "9C", "8C", "7C", "6C", "5C", "4C", "3C", "2C",
        "AH", "KH", "QH", "JH", "10H", "9H", "8H", "7H", "6H", "5H", "4H", "3H", "2H",
    ]

    shoe = deck_of_cards * num_of_decks
    return shoe

def deal_from_deck(current_cards, shoe, num_of_cards):
    for i in range(num_of_cards):
        drawn_card = random.sample(shoe, 1)
        current_cards += drawn_card
        shoe.remove(drawn_card[0])
    return current_cards, shoe

#create a function that counts a players hand, and evaluates ace
def count_hand(hand):
    hand_score = 0
    aces_present = 0
    aces_used = 0

    for card in hand:
        card = card[:-1]
        if card in ["K", "Q", "J"]:
            hand_score += 10
        elif card != 'A':
            hand_score += int(card)
        elif card == 'A':
            aces_present += 1
            hand_score += 11

    while hand_score > 21 and aces_present > 0 and aces_used != aces_present:
        hand_score -= 10
        aces_used += 1

    return hand_score

def check_for_split(player_hand): #use later
    return len(player_hand) == 2 and player_hand[0][0] == player_hand[1][0]

'''
def choose_action(number_of_actions = 0):
    #creating simple version 
    verified_action = None
    while verified_action == None:
        action = input("H, S") 
        if str(action)[0].lower() == 'h':
            verified_action = 'hit'
        elif str(action)[0].lower() == 's':
            verified_action = 'stand'
        else:
            print("Incorrect response, try again")
    return verified_action
'''
def choose_action(optimal_play_dict, card_count, dealer_card, has_ace):
    if card_count < 12:
        return 'hit'
    else:
        action_value = optimal_play_dict[str(has_ace)][str(card_count)][str(dealer_card)]

        print("player ace", has_ace, "player action_value", action_value)
        if action_value == 0:
            return 'stand'
        else:
            return 'hit'




def print_hand_state(player_hand, dealer_hand, pre_hand = True):
    print('Player Cards:' , player_hand)
    print("Player Count:" , count_hand(player_hand))
    if pre_hand == True:
        print('Dealer Shows:', dealer_hand[0][:-1])
    else:
        print('Dealer Cards:' , dealer_hand)
        print("Dealer Count:" , count_hand(dealer_hand))


def player_has_ace(player_hand):
    ace_present = 0
    for card in player_hand:
        if card == 'A':
            ace_present = 1
    return ace_present

'''
def win_condition(dealer_hand, player_hand):
    if player_hand > 21:
        return 0
    elif dealer_han
    elif player_hand <= 21 and dealer_hand 
'''

'\ndef win_condition(dealer_hand, player_hand):\n    if player_hand > 21:\n        return 0\n    elif dealer_han\n    elif player_hand <= 21 and dealer_hand \n'

In [102]:
#above this would require chips + ensuring shoe at least 25% full
def play_hand(shoe, print_cards = True):
    print("New Hand")
    dealer, shoe = deal_from_deck([], shoe, 2)
    dealer_first_card = dealer[0][:-1]
    player, shoe = deal_from_deck([], shoe, 2)
    print(player, dealer)

    while count_hand(player) < 21:
        if print_cards == True:  
            print_hand_state(player, dealer)
        action = choose_action(optimal_play_dict, count_hand(player), dealer_first_card, player_has_ace(player))
        if action == 'hit':
            player, shoe = deal_from_deck(player, shoe, 1)            
            print(player)
        elif action == 'stand':
            break

    #dealer action, no hit on 17
    print(dealer)
    while count_hand(dealer) < 21:
        dealer_score = count_hand(dealer)
        if dealer_score >= 17:
            break
        else:
            dealer, shoe = deal_from_deck(dealer, shoe, 1)

    if print_cards == True: 
        print("final score") 
        print_hand_state(player, dealer, pre_hand = False)

    #win condition
    dealer_score, player_score = count_hand(dealer), count_hand(player)
    if player_score > 21:
        return 0
    elif dealer_score > 21:
        return 2
    elif player_score < dealer_score:
        return 0
    elif player_score == dealer_score:
        return 1
    elif player_score > dealer_score:
        return 2


In [103]:
def session_init(num_of_decks):
    shoe = generate_deck(num_of_decks)
    shoe_at_max_size = len(shoe)

    win_count, tie_count, loss_count = 0,0,0

    while len(shoe) / shoe_at_max_size > .25:
        result = play_hand(shoe)
        if result == 2:
            win_count += 1
        elif result == 1:
            tie_count += 1
        elif result == 0:
            loss_count += 1
        print("Wins:" , win_count)
        print("Ties:" , tie_count)
        print("Losses:" , loss_count)

    print("Shoe empty, play again?")
    



In [104]:
session_init(5)

New Hand
['2D', '10S'] ['5S', '2C']
Player Cards: ['2D', '10S']
Player Count: 12
Dealer Shows: 5
player ace 0 player action_value 0
['5S', '2C']
final score
Player Cards: ['2D', '10S']
Player Count: 12
Dealer Cards: ['5S', '2C', '3H', 'AC']
Dealer Count: 21
Wins: 0
Ties: 0
Losses: 1
New Hand
['9S', '4D'] ['4C', 'AC']
Player Cards: ['9S', '4D']
Player Count: 13
Dealer Shows: 4
player ace 0 player action_value 0
['4C', 'AC']
final score
Player Cards: ['9S', '4D']
Player Count: 13
Dealer Cards: ['4C', 'AC', 'KC', '4C']
Dealer Count: 19
Wins: 0
Ties: 0
Losses: 2
New Hand
['JH', '3S'] ['2D', '3S']
Player Cards: ['JH', '3S']
Player Count: 13
Dealer Shows: 2
player ace 0 player action_value 0
['2D', '3S']
final score
Player Cards: ['JH', '3S']
Player Count: 13
Dealer Cards: ['2D', '3S', '10C', '7H']
Dealer Count: 22
Wins: 1
Ties: 0
Losses: 2
New Hand
['8S', '2H'] ['7C', '8H']
Player Cards: ['8S', '2H']
Player Count: 10
Dealer Shows: 7
['8S', '2H', '5H']
Player Cards: ['8S', '2H', '5H']
Player